In [4]:
import twitter
import tweepy
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import consumer_secret, consumer_key, access_token_key, access_token_secret

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [6]:
def twittercompany(handle):
    tweets = tweepy.Cursor(api.user_timeline, screen_name=handle, tweet_mode="extended").items()
    tweets_json = []
    for tweet in tweets:
        tweets_json.append(tweet)

    text=[]
    favorites = []
    retweets = []
    tweetdate = []
    for tweet in range(0,len(tweets_json)):
        #text
        textj = tweets_json[tweet].full_text
        text.append(textj)
        #favorites
        favoritesj = tweets_json[tweet].favorite_count
        favorites.append(favoritesj)
        #retweets
        retweetsj = tweets_json[tweet].retweet_count
        retweets.append(retweetsj)
        #date
        datej = tweets_json[tweet].created_at
        tweetdate.append(datej)

    frame = pd.DataFrame({"Text": text, "Favorites": favorites, "Retweets": retweets, "Date": tweetdate})

    frame["DOW"] = pd.to_datetime(frame["Date"]).dt.weekday
    frame = frame[frame.DOW < 5]

    frame["Date"] = pd.to_datetime(frame["Date"])
    frame["Date"] = frame["Date"].dt.date
    bydate = frame.groupby(["Date"]).sum()
    bydate['Datevalue'] = bydate.index.astype(str)
    bydate['Datevalue'] = bydate['Datevalue'].str.replace('\D', '').astype(int)
    
    return bydate

In [5]:
twitter = twittercompany("Nike")
twitter

,Favorites,Retweets,DOW,Datevalue
Date,,,,
2018-01-16,1,2,2,20180116
2018-01-17,20,3,14,20180117
2018-01-18,14,0,24,20180118
2018-01-19,11,1,32,20180119
2018-01-22,3,1,0,20180122
...,...,...,...,...
2019-12-09,5,3,0,20191209
2019-12-10,848,228,10,20191210
2019-12-11,18,2,8,20191211


In [7]:
import json
import requests
import pandas as pd
from config import stock_key
import pprint

In [8]:
def companystock(symbol):
    url = "https://www.alphavantage.co/"
    symbol = symbol
    query_url = url + "query?function=TIME_SERIES_DAILY&symbol=" + symbol +"&apikey=" + stock_key

    response = requests.get(url)
    stock_response = requests.get(query_url)
    stock_json = stock_response.json()

    stock_response = requests.get(query_url)

    stock_json = stock_response.json()
    stock_list =[]
    stock_info =[]
    for key, value in stock_json.items():
        stock_list.append([key,value])



    for key, value in stock_list[1][1].items():
        stock_day = []
        stock_day.append(key)
        for col, item in value.items():
            stock_day.append(item)
        stock_info.append(stock_day)

    df_stock_json = pd.DataFrame(stock_info)

    df_stock_json.columns =["Date", 'Open', 'High', 'Low', 'Close', 'Volume']
    
    close_list = []
    df_stock_json["Close"] =df_stock_json["Close"].astype(float)
    #df_stock_json["Close"] = (df_stock_json["Close"] - df_stock_json["Close"].mean())/df_stock_json["Close"].std()
    df_stock_json = df_stock_json.sort_values(by = "Date", ascending = True, )
    df_stock_json = df_stock_json.reset_index(drop=True)
    df_stock_json['Datevalue'] = df_stock_json['Date'].astype(str)
    df_stock_json['Datevalue'] = df_stock_json['Datevalue'].str.replace('\D', '').astype(int)

    return df_stock_json


In [8]:
stock = companystock("NKE")
stock

,Date,Open,High,Low,Close,Volume,Datevalue
0,2019-07-26,87.4400,87.8400,86.7700,87.54,3707249,20190726
1,2019-07-29,87.6500,87.8700,87.1800,87.62,4387337,20190729
2,2019-07-30,86.8200,87.9108,86.3600,87.20,3562999,20190730
3,2019-07-31,87.0900,87.2700,85.5500,86.03,5818168,20190731
4,2019-08-01,85.2600,86.7700,82.7000,83.12,7871849,20190801
...,...,...,...,...,...,...,...
95,2019-12-10,96.7600,97.5900,96.6300,97.01,4233906,20191210
96,2019-12-11,96.9000,97.3200,96.6500,97.18,4292584,20191211
97,2019-12-12,97.3200,98.1800,96.9000,97.72,4762665,20191212
98,2019-12-13,97.4300,98.2900,96.9100,97.77,5247551,20191213


In [9]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
public_gtweets = []
sentiment = []
means = []
def clean_tweet(tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
def get_tweet_sentiment(tweet): 
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_tweet(tweet)) 
        # set sentiment 
        return analysis.sentiment.polarity
import datetime
import numpy as np
def sentimentdaily(search, startyear, startmonth, startday, endyear, endmonth, endday):
    start_date=datetime.date(startyear,startmonth,startday)
    end_date=datetime.date(endyear,endmonth,endday)
    difference= (end_date-start_date).days+1
    dates = []
    for d in range(difference):
        date = start_date+datetime.timedelta(d)
        dates.append(date)
    dates
    publictweets = []
    for n in range(0, len(dates)-1):
        public = tweepy.Cursor(api.search,
        q=search,
        since=dates[n],
        until=dates[n+1],
        lang="en", tweet_mode = "extended").items(200)
        for tweet in public:
            publictweets.append(tweet.full_text)
        for tweet in range(1, len(publictweets)):
            sentiment.append(get_tweet_sentiment(clean_tweet(publictweets[tweet])))
        score = np.mean(sentiment)
        means.append(score)
    Sentimentbyday = pd.DataFrame({"Date": dates[:-1], "Sentiment": means})
    Sentimentbyday["DOW"] = pd.to_datetime(Sentimentbyday["Date"]).dt.weekday
    Sentimentbyday = Sentimentbyday[Sentimentbyday.DOW < 5]
    Sentimentbyday['Datevalue'] = Sentimentbyday['Date'].astype(str)
    Sentimentbyday['Datevalue'] = Sentimentbyday['Datevalue'].str.replace('\D', '').astype(int)

    return Sentimentbyday

In [10]:
sentiment = sentimentdaily("Nike", 2019, 12, 9, 2019, 12, 16)
sentiment

,Date,Sentiment,DOW,Datevalue
0,2019-12-09,0.034009,0,20191209
1,2019-12-10,0.064891,1,20191210
2,2019-12-11,0.077192,2,20191211
3,2019-12-12,0.080111,3,20191212
4,2019-12-13,0.082112,4,20191213


In [10]:
from pytrends.request import TrendReq
import time
import numpy as np
import random
def trends(search):
    pytrend = TrendReq(hl='en-US', tz=360)
    kw_list=[search]
    pytrend.build_payload(kw_list, cat=0, timeframe='today 3-m',geo='',gprop='')
    interest_over_time_df = pytrend.interest_over_time()
    time.sleep(random.randint(5, 10))

    interest_over_time_df["Date"] = interest_over_time_df.index

    interest_over_time_df.reset_index
    interest_over_time_df["DOW"] = pd.to_datetime(interest_over_time_df["Date"]).dt.weekday
    interest_over_time_df = interest_over_time_df[interest_over_time_df.DOW < 5]
    interest_over_time_df['Datevalue'] = interest_over_time_df['Date'].astype(str)
    interest_over_time_df['Datevalue'] = interest_over_time_df['Datevalue'].str.replace('\D', '').astype(int)


    return interest_over_time_df

In [12]:
trends = trends("Nike")


In [13]:
trends

,Nike,isPartial,Date,DOW,Datevalue
date,,,,,
2019-09-16,40,False,2019-09-16,0,20190916
2019-09-17,40,False,2019-09-17,1,20190917
2019-09-18,39,False,2019-09-18,2,20190918
2019-09-19,40,False,2019-09-19,3,20190919
2019-09-20,42,False,2019-09-20,4,20190920
...,...,...,...,...,...
2019-12-09,52,False,2019-12-09,0,20191209
2019-12-10,51,False,2019-12-10,1,20191210
2019-12-11,50,False,2019-12-11,2,20191211


In [11]:
df1 = twitter.merge(stock, on = "Datevalue")
df2 = df1.merge(sentiment, on ="Datevalue")
total = df2.merge(trends, on = "Datevalue")
total = total.drop(["DOW_x", "Date_x", "isPartial", "Date_y", "DOW_y"], axis = 1)

AttributeError: module 'twitter' has no attribute 'merge'

In [18]:
total["Retweetsdb"] = total["Retweets"].shift(1)
total["Favoritesdb"] = total["Favorites"].shift(1)
total["Googledb"] = total["Nike"].shift(1)
total["Sentimentdb"] = total["Sentiment"].shift(1)
total = total.reset_index()
total

,index,Favorites,Retweets,Datevalue,Open,High,Low,Close,Volume,Sentiment,Nike,Date,DOW,Retweetsdb,Favoritesdb,Googledb,Sentimentdb
0,0,5,3,20191209,97.0200,97.6300,96.1600,96.63,6517768,0.034009,52,2019-12-09,0,NaN,NaN,NaN,NaN
1,1,848,228,20191210,96.7600,97.5900,96.6300,97.01,4233906,0.064891,51,2019-12-10,1,3.0,5.0,52.0,0.034009
2,2,18,2,20191211,96.9000,97.3200,96.6500,97.18,4292584,0.077192,50,2019-12-11,2,228.0,848.0,51.0,0.064891
3,3,8,0,20191213,97.4300,98.2900,96.9100,97.77,5247551,0.082112,54,2019-12-13,4,2.0,18.0,50.0,0.077192


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt   #Data visualisation libraries 
import seaborn as sns
%matplotlib inline

In [20]:
total.corr()

,index,Favorites,Retweets,Datevalue,Close,Sentiment,Nike,DOW,Retweetsdb,Favoritesdb,Googledb,Sentimentdb
index,1.000000,-0.253039,-0.268069,0.982708,0.976875,0.935549,0.377964,0.982708,-0.003840,0.013458,-1.000000,0.970500
Favorites,-0.253039,1.000000,0.999847,-0.289948,-0.190583,0.018553,-0.302064,-0.289948,-0.487642,-0.502671,0.871164,-0.963844
Retweets,-0.268069,0.999847,1.000000,-0.303100,-0.203856,0.001531,-0.299651,-0.303100,-0.490034,-0.505039,0.869815,-0.963110
Datevalue,0.982708,-0.289948,-0.303100,1.000000,0.994540,0.865809,0.542857,1.000000,-0.192752,-0.175750,-0.981981,0.907447
Close,0.976875,-0.190583,-0.203856,0.994540,1.000000,0.881072,0.542009,0.994540,-0.307621,-0.291117,-0.952683,0.851292
Sentiment,0.935549,0.018553,0.001531,0.865809,0.881072,1.000000,0.085793,0.865809,0.236485,0.253257,-0.970720,1.000000
Nike,0.377964,-0.302064,-0.299651,0.542857,0.542009,0.085793,1.000000,0.542857,-0.696137,-0.683615,-0.720577,0.532145
DOW,0.982708,-0.289948,-0.303100,1.000000,0.994540,0.865809,0.542857,1.000000,-0.192752,-0.175750,-0.981981,0.907447
Retweetsdb,-0.003840,-0.487642,-0.490034,-0.192752,-0.307621,0.236485,-0.696137,-0.192752,1.000000,0.999850,0.003840,0.237374
Favoritesdb,0.013458,-0.502671,-0.505039,-0.175750,-0.291117,0.253257,-0.683615,-0.175750,0.999850,1.000000,-0.013458,0.254142


In [22]:
import statsmodels.api as sm # import statsmodels 
Y = total["Close"]
Y = Y.drop(Y.index[0])
X = total[["Favorites", "Retweets", 'Favoritesdb', 'Retweetsdb', "Nike", "Googledb"]]
X = X.drop(X.index[0])
model = sm.OLS(Y, X).fit()
model.summary()

C:\Users\Owner\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1648: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\Owner\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1649: RuntimeWarning: invalid value encountered in double_scalars
  * (1 - self.rsquared))
C:\Users\Owner\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid
C:\Users\Owner\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1578: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Mon, 16 Dec 2019   Prob (F-statistic):                nan
Time:                        19:02:39   Log-Likelihood:                 90.358
No. Observations:                   3   AIC:                            -174.7
Df Residuals:                       0   BIC:                            -177.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Favorites       0.0105        inf          0        nan         nan         nan
Retweets       -0.0359        inf         -0        nan         nan         nan
Favoritesdb     0.0161        inf          0        nan         nan         nan
Retweetsdb     -0.0485        inf         -0        nan         nan         nan
Nike            0.9733        inf          0        nan         nan         nan
Googledb        0.8987        inf          0        nan         nan         nan
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.167
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                          -0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         12.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The input rank is higher than the number of observations.
"""